<a href="https://colab.research.google.com/github/marcelalozano27-ship-it/machine-learning-2026-spring/blob/main/CA-02/CA02_NB_assignment_BR_ML_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CA-02: Email Spam Classifier using Naive Bayes
#### Completed by: Marcela and Brandon



In this assignment you will ...
1. Complete the code such a way that it works correctly with this given parts of the program.
2. Explain as clearly as possible what each part of the code is doing. Use "Markdown" texts and code commenting to explain the code

IMPORTANT NOTE:

The path of your data folders 'train-mails' and 'test-mails' must be './train-mails' and './test-mails'. This means you must have your .ipynb file and these folders in the SAME FOLDER in your laptop or Google Drive. The reason for doing this is, this way the peer reviewes and I would be able to run your code from our computers using this exact same relative path, irrespective of our folder hierarchy.

In [1]:
import os
import numpy as np
from collections import Counter

#We start by importing the required libraries to be able to run the Naive Bayes model
import numpy as np
import pandas as pd

#The sklearn package has tools for splitting the data, training the Naive Bayes model, and testing the model accuracy
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
#Mount drive to allow the program to access the testing and training email data folders (The files are very large so this is the best way to read files while running the program in Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Dictionary Function
###In this first function we will create a dictionary for the various words within the emails as we look to detect which are spam or not. To start the dictionary will collect all the words (all_words[]) in the e-mails and going line by line split the words apart and add it to the overall word list. The the Counter method will count every instance of each word.
###The next loop in this function then helps eliminate words that are not useful. Using .isalpha() function goes through each word and if it has a number or punctutation in it then it will delete it from the record. The other criteria is if the word is equal to one (such as a, i, etc.) then it will delete those records as well.
###Finally, the function will create a dictionary that takes the most common 3000 words to use later on for the spam detection model.

In [3]:

def make_Dictionary(root_dir):
  all_words = []
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
#this goes through every email and splits the words into a vocabulary list (bag of words)
  for mail in emails:
    with open(mail) as m:
      for line in m:
        words = line.split()
        all_words += words
  dictionary = Counter(all_words)
  list_to_remove = list(dictionary)
#this loop deletes the numbers, punctuations, and words that are just one letter
  for item in list_to_remove:
    if item.isalpha() == False:
      del dictionary[item]
    elif len(item) == 1:
      del dictionary[item]
#the function returns the most common 3000 words
  dictionary = dictionary.most_common(3000)
  return dictionary


#Extract Features Function
###Next we will define a function called extract_features that will convert each email into a numerical feature vector using the top 3,000 words. Each email is represented as a vector of length 3,000 where each position corresponds to a word from the dictionary and the value stored is a count of each occurrence of the word. These rows of email vectors form the feature matrix which is our numerical representation of the dataset used for training the machine.

###The feature matrix lets the Naive Bayes classifier calculate the probability of an email being spam or not based on the distribution of words across the different emails.

###Finally the function will identify file names starting with "spmsg" to correctly return a 1 for spam emails and 0 for non spam emails.

In [4]:

def extract_features(mail_dir):
#Collect the files
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
  features_matrix = np.zeros((len(files),3000))
  train_labels = np.zeros(len(files))
  count = 1;
  docID = 0;
#Loop through each email file
  for fil in files:
    with open(fil) as fi:
#Extracting words from the email text using only the third line as the body of the text.
      for i, line in enumerate(fi):
        if i ==2:
          words = line.split()
#Convert each word into numerical features to build the Bag of words
          for word in words:
            wordID = 0
            for i, d in enumerate(dictionary):
              if d[0] == word:
                wordID = i
                features_matrix[docID,wordID] = words.count(word)
#Assign spam labels based on the filename. Files starting with spmsg will be assigned a 1 and all other files will be assigned a 0. This will form our features matrix and label vector.
      train_labels[docID] = 0;
      filepathTokens = fil.split('/')
      lastToken = filepathTokens[len(filepathTokens)-1]
      if lastToken.startswith("spmsg"):
        train_labels[docID] = 1;
        count = count + 1
      docID = docID + 1
  return features_matrix, train_labels


### Instructions for using Google Colab
1. Upload notebook and data folders to Google Drive
2. Open notebook in Colab
3. Mount drive and change path to your Google Drive folder

In [5]:
#PATH to "train_mails" and "test-mails"
TRAIN_DIR = "/content/drive/MyDrive/CA-02/Data/train-mails"
TEST_DIR = "/content/drive/MyDrive/CA-02/Data/test-mails"


In [6]:
#Create a dictionary with the previously created dictionary function using the most frequent 3,000 wordsin the training data to define feature space
dictionary = make_Dictionary(TRAIN_DIR)

print ("reading and processing emails from TRAIN and TEST folders")

#Use the extract features function with the words in the dictionary to create a feature matrix and assign each training email a 1 for spam and 0 for non spam
train_features_matrix, train_labels = extract_features(TRAIN_DIR)
#Extract features from test emails using the same dictionary for consistency. Spam labels are also assigned a 1 and non spam are assigned a 0 using the same labeling as the training data
test_features_matrix, test_labels = extract_features(TEST_DIR)

reading and processing emails from TRAIN and TEST folders


In [7]:
# In this section enter your code to TRAIN the model using Naive Bayes algorithm, then PREDICT and then evaluate PERFORMANCE (Accuracy)
#Initialize the Naive Bayes Model we imported earlier
model=GaussianNB()
#Train the model using the training feature matrix and corresponding spam/not spam labels. The model learns from the patterns of word distributions for emails labeled spam vs not spam
print("Training Model using Gaussian Naive Bayes algorithm...")
model.fit(train_features_matrix, train_labels)
print("Training Completed")
print("Testing trained model to predict Test data labels")
#After the model is trained on the training data, it generates predictions for each email in the test dataset labeled spam or not spam.
modelpredictions=model.predict(test_features_matrix)
#The model accuracy is stored as a comparison of the model's predicted labels from the test data and the true labels from the test set. The accuracy gives the percentage of correctly classified emails
modelaccuracy=accuracy_score(test_labels, modelpredictions)
print("Completed classification of the test data...now printing accuracy score by comparing the predicted labels with the test labels.")
print(modelaccuracy)

Training Model using Gaussian Naive Bayes algorithm...
Training Completed
Testing trained model to predict Test data labels
Completed classification of the test data...now printing accuracy score by comparing the predicted labels with the test labels.
0.9615384615384616


======================= END OF PROGRAM =========================